# Scatterplots
## Via Holoviews


### Preamble:

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import holoviews as hv
hv.extension('bokeh')
renderer = hv.renderer('bokeh')
renderer.size = 150

### Data

In [ ]:
df = pd.read_csv('../data/iris.csv')
df.drop(columns='species_code', inplace=True)
df.head()

## Scatter Plots, declarative style
### Graph-elements, not pixel-maps
First things first; a basic scatterplot transform. We pass in a data source (a dataframe in this case) and the 'kdims', the key-dimensions that we'll use to define the x and y coord for each point.

In [ ]:
points = hv.Points(df, kdims=['sepal_width', 'petal_length']).options(size=5)
points

Note, we can change properties on the fly; this is data-with-representation, not a pixel-map:

In [ ]:
points.options(size=10, color='orange')

How does that work? By attaching the data to the plot element...

In [ ]:
points.data.head()

Note that we can explictly choose other columns as 'vdims', values that will be attached to each point, like so:

In [ ]:
_ = hv.Points(df, kdims=['sepal_width', 'petal_length'], vdims=['species'])

By default, all non-key dims are stored as value dims:

In [ ]:
df.agg(('min','max'))

And we can use these to control the properties of each point...

In [ ]:
%opts Points (cmap='tab10' size=15)  [tools=['hover']]
points.options(color_index='species', size_index='petal_width')

## Building layouts
OK, we've probably stretched the data-content of a single axis as far as is sensible.
Now, let's pull together subplots and **plot all the things!**

In [ ]:
import itertools
pairings = list(itertools.combinations(df.columns[:4], 2))
pairings

In [ ]:
all_the_things = [
    hv.Points(df, kdims=list(p)[::-1]).options(
        color_index='species').relabel(f'{p[1]} vs {p[0]}')
    for p in pairings
]

In [ ]:
all_the_things

In [ ]:
%output size=100

In [ ]:
%%opts Points [tools=['hover', 'lasso_select'] fontsize={'legend':8} show_legend=False  padding=0.1]
%%opts Points (size=6 alpha=0.7 selection_alpha=1. nonselection_alpha=0.1)
# %%opts Points [tools=['hover', 'lasso_select'] fontsize={'legend':8} show_legend=False  ] (size=6)
layout=hv.Layout(all_the_things).options(shared_datasource=True).cols(3).relabel('The Iris Dataset, Holoviews style')
layout

## Exporting
Note, this isn't only interactive in the notebook. We can save it as an interactive, standalone HTML plot -though this might get large, since it stores all the source-data!

In [ ]:
renderer.save(layout, 'all_the_iris')

### Footnotes: padding, alpha
 * Note that you need to be running the master (dev!) version of holoviews for the `padding` keyword to work, as it was only merged in 6 days ago at time of writing:
https://github.com/ioam/holoviews/pull/2293

* If you tweak the `alpha` transparency of the Points, then **selection will appear to stop working** unless you also tweak the various `selection_alpha` and `nonselection_alpha` options, cf [this issue](https://github.com/ioam/holoviews/issues/2600#issuecomment-384715709).

Still a few rough edges!